# LightGBM Demand Forecasting Training Pipeline

This notebook trains and evaluates LightGBM models for demand forecasting on the smartSTAT medication datasets.

## Configuration

Set the data directory and other parameters at the top of the notebook.


In [ ]:
# Configuration - Update these paths as needed
import sys
from pathlib import Path

# Get notebook directory and project root
NOTEBOOK_DIR = Path().resolve()  # ml/notebooks/
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent  # project root

# Add project root to path to import ml modules
sys.path.insert(0, str(PROJECT_ROOT))

# Data directory (update this if moving data to cloud storage)
# Default: lib/smartstat_synth_2023-2025 relative to project root
DATA_DIR = PROJECT_ROOT / "lib" / "smartstat_synth_2023-2025"

# Output directory for models and results (relative to project root)
OUTPUT_DIR = PROJECT_ROOT / "ml" / "models"

# Dataset ID to train (e.g., "A1", "E10", or use "all" to loop through all)
DATASET_ID = "A1"

# Forecast horizon (days): 7, 14, or 30
HORIZON = 14

# Model objective: "l2" (regression) or "quantile" (quantile regression)
OBJECTIVE = "l2"

# Quantile alpha (only used if OBJECTIVE == "quantile")
QUANTILE_ALPHA = 0.95

# Exclude contemporaneous used_units from features (use only lags)
EXCLUDE_CONTEMPORANEOUS_USED = False

print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Dataset ID: {DATASET_ID}")
print(f"Horizon: {HORIZON} days")
print(f"Objective: {OBJECTIVE}")


In [ ]:
# Imports
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Optional, Tuple
import json

from ml.config import (
    DEFAULT_LIGHTGBM_PARAMS,
    VALIDATION_SPLIT_DAYS,
    RANDOM_SEED,
    EARLY_STOPPING_ROUNDS,
)
from ml.datasets import (
    load_dataset_files,
    get_all_dataset_ids,
    get_dataset_info,
)
from ml.features import (
    prepare_features_and_labels,
    create_time_based_validation_split,
)
from ml.evaluate import (
    compute_metrics,
    save_predictions,
    save_metrics,
)

# Set style for plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set random seeds for reproducibility
np.random.seed(RANDOM_SEED)

print("Imports complete!")


## Load Dataset

Load the training and test data for the selected dataset.


In [ ]:
# Get dataset info
dataset_info = get_dataset_info(DATA_DIR, DATASET_ID)
print(f"Dataset Info:")
print(f"  Type: {dataset_info['type']}")
print(f"  Avg used (train): {dataset_info['avg_used_train']:.2f}")
print(f"  Avg used (test): {dataset_info['avg_used_test']:.2f}")
print(f"  Max used (train): {dataset_info['max_used_train']}")
print(f"  Lead time: {dataset_info['lead_time_days']} days")

# Load datasets
train_df, test_df = load_dataset_files(DATA_DIR, DATASET_ID)

print(f"\nLoaded datasets:")
print(f"  Train: {len(train_df)} rows ({train_df['date'].min()} to {train_df['date'].max()})")
print(f"  Test: {len(test_df)} rows ({test_df['date'].min()} to {test_df['date'].max()})")

# Display first few rows
print(f"\nTrain data preview:")
train_df.head()


In [ ]:
# Visualize usage patterns
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Training data
axes[0].plot(train_df['date'], train_df['used_units'], alpha=0.7, linewidth=0.8)
axes[0].set_title(f'Training Data: Daily Usage - Dataset {DATASET_ID}')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Used Units')
axes[0].grid(True, alpha=0.3)

# Test data
axes[1].plot(test_df['date'], test_df['used_units'], alpha=0.7, linewidth=0.8, color='orange')
axes[1].set_title(f'Test Data: Daily Usage - Dataset {DATASET_ID}')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Used Units')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\nUsage Statistics:")
print(f"  Train - Mean: {train_df['used_units'].mean():.2f}, Std: {train_df['used_units'].std():.2f}")
print(f"  Test  - Mean: {test_df['used_units'].mean():.2f}, Std: {test_df['used_units'].std():.2f}")


## Prepare Features and Labels

Create labels for multi-horizon forecasting and prepare feature sets. Labels are created using only future data (no leakage).


In [ ]:
# Prepare features and labels
X_train, y_train, X_test, y_test = prepare_features_and_labels(
    train_df,
    test_df,
    horizon=HORIZON,
    exclude_contemporaneous_used=EXCLUDE_CONTEMPORANEOUS_USED,
)

print(f"Features ({len(X_train.columns)}):")
print(f"  {', '.join(X_train.columns.tolist())}")

print(f"\nDataset sizes after label creation:")
print(f"  Train: {len(X_train)} samples (last {HORIZON} rows dropped)")
print(f"  Test: {len(X_test)} samples (last {HORIZON} rows dropped)")

print(f"\nLabel statistics (sum of next {HORIZON} days):")
print(f"  Train - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}, Min: {y_train.min():.2f}, Max: {y_train.max():.2f}")
print(f"  Test  - Mean: {y_test.mean():.2f}, Std: {y_test.std():.2f}, Min: {y_test.min():.2f}, Max: {y_test.max():.2f}")

# Check for missing values
print(f"\nMissing values in features:")
missing = X_train.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("  None!")


In [ ]:
# Create time-based validation split
X_train_split, y_train_split, X_val, y_val = create_time_based_validation_split(
    X_train, y_train, validation_days=VALIDATION_SPLIT_DAYS
)

print(f"Time-based validation split:")
print(f"  Train split: {len(X_train_split)} samples")
print(f"  Validation: {len(X_val)} samples (last {VALIDATION_SPLIT_DAYS} days)")

# Visualize label distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].hist(y_train_split, bins=50, alpha=0.7, label='Train')
axes[0].hist(y_val, bins=50, alpha=0.7, label='Validation')
axes[0].set_xlabel(f'Label (sum of next {HORIZON} days)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Label Distribution: Train vs Validation')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].hist(y_test, bins=50, alpha=0.7, color='orange')
axes[1].set_xlabel(f'Label (sum of next {HORIZON} days)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Label Distribution: Test Set')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## Train LightGBM Model

Train the model with early stopping on the validation set.


In [ ]:
# Configure model parameters
params = DEFAULT_LIGHTGBM_PARAMS.copy()

if OBJECTIVE == "quantile":
    params["objective"] = "quantile"
    params["alpha"] = QUANTILE_ALPHA
    print(f"Using quantile regression with alpha={QUANTILE_ALPHA}")
else:
    params["objective"] = "regression"
    params["metric"] = "rmse"
    print(f"Using L2 regression (RMSE)")

print(f"\nModel parameters:")
for key, value in params.items():
    print(f"  {key}: {value}")

# Create LightGBM datasets
train_data = lgb.Dataset(X_train_split, label=y_train_split)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

print(f"\nTraining model...")
# Train model
model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, val_data],
    valid_names=["train", "val"],
    num_boost_round=1000,
    callbacks=[
        lgb.early_stopping(early_stopping_rounds=EARLY_STOPPING_ROUNDS, verbose=True),
        lgb.log_evaluation(period=50),
    ],
)

print(f"\nTraining complete!")
print(f"Best iteration: {model.best_iteration}")
print(f"Best score: {model.best_score}")


In [ ]:
# Plot training history
train_metric = list(model.best_score['train'].values())[0]
val_metric = list(model.best_score['val'].values())[0]

print(f"Final metrics:")
print(f"  Train {train_metric}: {model.best_score['train'][train_metric]:.4f}")
print(f"  Val {val_metric}: {model.best_score['val'][val_metric]:.4f}")

# Feature importance
feature_importance = pd.DataFrame({
    "feature": X_train.columns,
    "importance": model.feature_importance(importance_type="gain"),
}).sort_values("importance", ascending=False)

print(f"\nTop 10 most important features:")
print(feature_importance.head(10))

# Plot feature importance
plt.figure(figsize=(10, 6))
top_n = min(15, len(feature_importance))
sns.barplot(data=feature_importance.head(top_n), x="importance", y="feature", palette="viridis")
plt.title(f"Feature Importance (Top {top_n})")
plt.xlabel("Importance (Gain)")
plt.tight_layout()
plt.show()


## Evaluate on Test Set

Make predictions on the test set and compute evaluation metrics.


In [ ]:
# Make predictions on test set
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# Compute metrics
metrics = compute_metrics(y_test, pd.Series(y_pred))

print(f"Test Set Metrics:")
print(f"  MAE: {metrics['mae']:.2f}")
print(f"  RMSE: {metrics['rmse']:.2f}")
if metrics['mape']:
    print(f"  MAPE: {metrics['mape']:.2f}% (calculated on {metrics['mape_count']}/{metrics['total_samples']} samples)")
else:
    print(f"  MAPE: N/A (insufficient non-zero targets)")

# Create predictions dataframe for visualization
test_dates = test_df["date"].iloc[:len(y_test)]
predictions_df = pd.DataFrame({
    "date": test_dates,
    "actual": y_test.values,
    "predicted": y_pred,
    "error": y_test.values - y_pred,
    "abs_error": np.abs(y_test.values - y_pred),
})

predictions_df.head(10)


In [ ]:
# Visualize predictions vs actuals
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Time series plot
axes[0].plot(predictions_df['date'], predictions_df['actual'], 
             label='Actual', alpha=0.7, linewidth=1.5, color='blue')
axes[0].plot(predictions_df['date'], predictions_df['predicted'], 
             label='Predicted', alpha=0.7, linewidth=1.5, color='red', linestyle='--')
axes[0].set_xlabel('Date')
axes[0].set_ylabel(f'Label (sum of next {HORIZON} days)')
axes[0].set_title(f'Predictions vs Actuals - Dataset {DATASET_ID} (H={HORIZON})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals plot
axes[1].scatter(predictions_df['predicted'], predictions_df['error'], 
                alpha=0.5, s=20)
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1)
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Residual (Actual - Predicted)')
axes[1].set_title('Residual Plot')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Scatter plot: predicted vs actual
plt.figure(figsize=(8, 8))
plt.scatter(predictions_df['actual'], predictions_df['predicted'], alpha=0.5, s=20)
min_val = min(predictions_df['actual'].min(), predictions_df['predicted'].min())
max_val = max(predictions_df['actual'].max(), predictions_df['predicted'].max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect prediction')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title(f'Predicted vs Actual - Dataset {DATASET_ID} (H={HORIZON})')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Error distribution
plt.figure(figsize=(10, 4))
plt.hist(predictions_df['error'], bins=50, alpha=0.7, edgecolor='black')
plt.axvline(x=0, color='red', linestyle='--', linewidth=2)
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.title('Error Distribution')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## Save Model and Results

Save the trained model, metrics, predictions, and feature importance to disk.


In [ ]:
# Create output directory
model_dir = OUTPUT_DIR / DATASET_ID
model_dir.mkdir(parents=True, exist_ok=True)

# Save model
model_filename = f"model_h{HORIZON}_{OBJECTIVE}"
if OBJECTIVE == "quantile":
    model_filename += f"_q{QUANTILE_ALPHA}"
model_filename += ".txt"
model_path = model_dir / model_filename
model.save_model(str(model_path))
print(f"✓ Saved model to: {model_path}")

# Save feature importance
importance_path = model_dir / f"feature_importance_h{HORIZON}_{OBJECTIVE}.csv"
feature_importance.to_csv(importance_path, index=False)
print(f"✓ Saved feature importance to: {importance_path}")

# Save predictions
predictions_path = model_dir / f"predictions_h{HORIZON}_{OBJECTIVE}.csv"
predictions_df.to_csv(predictions_path, index=False)
print(f"✓ Saved predictions to: {predictions_path}")

# Save metrics
metrics_path = model_dir / f"metrics_h{HORIZON}_{OBJECTIVE}.json"
save_metrics(
    metrics,
    metrics_path,
    DATASET_ID,
    HORIZON,
    OBJECTIVE,
    model.params,
    list(X_train.columns),
    (str(train_df["date"].min()), str(train_df["date"].max())),
    (str(test_df["date"].min()), str(test_df["date"].max())),
    QUANTILE_ALPHA if OBJECTIVE == "quantile" else None,
)
print(f"✓ Saved metrics to: {metrics_path}")

print(f"\nAll results saved to: {model_dir}")


## Summary

Training and evaluation complete! The model has been saved along with all metrics and predictions.

### Next Steps

- Try different horizons (7, 14, 30 days)
- Experiment with quantile regression for uncertainty estimation
- Train on multiple datasets using the CLI script: `python -m ml.train_lgbm --dataset_id all`
- Adjust hyperparameters in the configuration section
